### A Sketch-based Index for Correlated Dataset Search

Discovering Data with joinable keys and correlated data.

Steps of the algorithm:
1. creating a sketch of all tables in the Database
2. creating a sketch of the query table
3. find correlated and joinable tables for the query-table
(finding set overlap between the two sketches ?)

Sketch creation:
1. all tables with more then 2 columns are split:
    - all numerical columns are combined with all columns containig categorical values
2. all numeric values (k) per table are hashed and stored as tuple with their categorical value c_k
    - <h(k), c_k>
3. picking a specific sample (=sketch) of size n per table, using n tuples with smallest hash-value
    - this way the samples are comparable


correlation:


joinability:
    

In [18]:
# Algorithm

import pandas as pd

def hash(k):
    return k/1000

In [34]:
### Query dataset (Q)

df_Q = pd.DataFrame({'movies':['A','B','C','D','E'], 
                     'budget in mil €':[100, 200, 500, 300, 300],
                     'stars':[2,3,4,4.8,3]})
display(df_Q)

print()
print('hashed df:')
h_k = df_Q['budget in mil €'].apply(lambda k: hash(k))
df_Q['budget in mil €'] = h_k

h_k = df_Q['stars'].apply(lambda k: hash(k))
df_Q['stars'] = h_k

display(df_Q)

,movies,budget in mil €,stars
0,A,100,2.0
1,B,200,3.0
2,C,500,4.0
3,D,300,4.8
4,E,300,3.0



hashed df:


,movies,budget in mil €,stars
0,A,0.1,0.0020
1,B,0.2,0.0030
2,C,0.5,0.0040
3,D,0.3,0.0048
4,E,0.3,0.0030


In [36]:
### example of a corrolated and joinable dataset from the corpus (c):

df_c = pd.DataFrame({'movies':['C','D','E','F','G',], 
                     'budget per staff':[1.2, 3.5, 8, 10, 4]})
display(df_c)

h_k = df_c['budget per staff'].apply(lambda k: hash(k))
df_c['budget per staff'] = h_k
display(df_c)
#df = pd.DataFrame({'movies':['C','D','E','F','G',], 'sick days':[500, 50, 150, 175, 100]})
#df

,movies,budget per staff
0,C,1.2
1,D,3.5
2,E,8.0
3,F,10.0
4,G,4.0


,movies,budget per staff
0,C,0.0012
1,D,0.0035
2,E,0.0080
3,F,0.0100
4,G,0.0040


In [ ]:
# choose sketch for each table

In [ ]:
# find corrolated sketches and joinable sketches